In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# reshape the data as a matrix
def reshape_data(dataset, N, M):
    data = np.zeros((N, M))
    with open(dataset) as f:
        for line in f:
            user, item, record, timestamp = line.split()
            data[int(user) - 1][int(item) - 1] = float(record)
    return data


# mean absolute error
def MAE(R_origin, P, Q, N, M, K, beta):
    error = 0.0
    for i in range(N):
        for j in range(M):
            if R_origin[i][j] > 0:
                error += np.abs(R_origin[i][j] - np.dot(P[i, :], Q[:, j]))
    error = error / len(R_origin[R_origin != 0])
    return error

# hpyer-parameter 
N = 943
M = 1682
K = 10

epochs = 100
alpha = 0.0025
beta = 0.025

training_data = reshape_data(dataset='./u1.base', N=N, M=M)
testing_data = reshape_data(dataset='./u1.test', N=N, M=M)

# initialize matrix P and Q
P = np.random.rand(N, K)
Q = np.random.rand(K, M)

R = training_data
R_test = testing_data

train_error_list = []
test_error_list = []

# calculate the training error
train_error = MAE(R, P, Q, N, M, K, beta)
train_error_list.append(train_error)

# calculate the testing error
test_error = MAE(R_test, P, Q, N, M, K, beta)
test_error_list.append(test_error)

for epoch in range(epochs):
    for i in range(N):
        for j in range(M):
            if R[i][j] > 0:
                eij = R[i][j] - np.dot(P[i, :], Q[:, j])
                for k in range(K):
                    P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                    Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])

    # calculate the training error
    train_error = MAE(R, P, Q, N, M, K, beta)
    train_error_list.append(train_error)

    # calculate the testing error
    test_error = MAE(R_test, P, Q, N, M, K, beta)
    test_error_list.append(test_error)

# plot 
x = np.array(range(1, len(train_error_list) + 1))
plt.plot(x, np.array(train_error_list), label="Training Error")
plt.plot(x, np.array(test_error_list), label="Validation Error")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Experiment")
plt.legend()
plt.show()
